In [1]:

import sys
import os
import time
import numpy as np
import pandas as pd
import pickle
import re
from collections import defaultdict
import random
import random
import csv
import sqlite3 as lite
import unicodedata

In [14]:
start_time = time.clock()

chunk_size = 1000000
tsv_path = "../res/data.tsv"
# the columns in the tsv source
header_names = ["q_id","question","answer","label","a_id"]

#Will be created if doesnot exists
db_path  = "data.db"
table_name = "dataset"

con = None

try:

    file_read = 0
    con = lite.connect(db_path)

    for chunk in pd.read_csv(tsv_path,sep='\t',chunksize=chunk_size, header=None, names=header_names, encoding='utf8'):
        file_read+=chunk.shape[0]
        print(file_read, "\t Done")
        chunk.to_sql(name=table_name, con=con, if_exists='append', index=False)

except lite.Error:
    print("SQLite Error")
    sys.exit(1)

finally:
    if con:
        con.close()

print("--- %s seconds ---" % (time.clock() - start_time))

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


1000000 	 Done
2000000 	 Done
3000000 	 Done
4000000 	 Done
5000000 	 Done
5241880 	 Done
--- 123.32182353499957 seconds ---


D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [15]:
'''
BM25 is only a point to start with. 
Ideally the samples should be selected in a more sophisticated manner.
'''
#print(os.path.basename(__file__), "\n\n")
start_time = time.clock()

docIDFDict = {}
avgDocLength = 0
docIDFDict_file    = "../res/docIDFDict.pickle"
avgDocLength_file  = "../res/avgDocLength.pickle"

fileObject = open(docIDFDict_file, 'rb')
docIDFDict = pickle.load(fileObject)
fileObject.close()

fileObject = open(avgDocLength_file, 'rb')
avgDocLength = pickle.load(fileObject)
fileObject.close()

def GetBM25Score(Query, Passage, k1=1.5, b=0.75, delimiter=' ') :
    global docIDFDict,avgDocLength

    #remove special characters from query and passage
    Query = re.sub(r"[^a-zA-Z0-9]+", ' ', Query )
    Passage = re.sub(r"[^a-zA-Z0-9]+", ' ', Passage)

    query_words= Query.strip().lower().split(delimiter)
    passage_words = Passage.strip().lower().split(delimiter)
    passageLen = len(passage_words)
    docTF = {}
    for word in set(query_words):   #Find Term Frequency of all query unique words
        docTF[word] = passage_words.count(word)
    commonWords = set(query_words) & set(passage_words)
    tmp_score = []
    for word in commonWords :
        numer = (docTF[word] * (k1+1))   #Numerator part of BM25 Formula
        denom = ((docTF[word]) + k1*(1 - b + b*passageLen/avgDocLength)) #Denominator part of BM25 Formula
        if(word in docIDFDict) :
            tmp_score.append(docIDFDict[word] * numer / denom)
    score = sum(tmp_score)
    return score


print("BM25 Initialized")

chunk_size = 1000000
header_names = ["q_id","question","answer","label","a_id"]
bm25_header_names = ["q_id","question","answer","label","a_id","bm25_score"]
#Will be created if doesnot exists
db_path  = "data.db"
table_name = "dataset"
bm25_table_name = "dataset_bm25"

sql_read_query = "SELECT * FROM "+table_name

con = None
try:
    file_read = 0
    con = lite.connect(db_path)
    print("SQLite connection establshed")

    for chunk in pd.read_sql_query(sql=sql_read_query, con=con,chunksize=chunk_size):
        chunk = chunk.reindex(columns=bm25_header_names)
        chunk["bm25_score"] = chunk.apply(lambda x: GetBM25Score(x.question, x.answer), axis=1)
        chunk.to_sql(name=bm25_table_name, con=con, if_exists='append', index=False)
        file_read += chunk.shape[0]
        print(file_read, "\t Done")

    cursor = con.cursor()
    cursor.execute('''DROP TABLE dataset''')
    con.commit()

except lite.Error as e:
    print("SQLite Error")
    print(e)
    sys.exit(1)

finally:
    if con:
        con.close()

print("--- %s seconds ---" % (time.clock() - start_time))

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


BM25 Initialized
SQLite connection establshed
1000000 	 Done
2000000 	 Done
3000000 	 Done
4000000 	 Done
5000000 	 Done
5241880 	 Done


D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:83: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


--- 791.7546169649995 seconds ---


In [16]:

start_time = time.clock()
curr_time = time.clock()

chunk_size = 100000 #because 10 answers for a question
bm25_header_names = ["q_id","question","answer","label","a_id","bm25_score"]
db_path  = "data.db"
rearranged_table_name = "dataset_rearranged"
table_name = "dataset_bm25"

total_rows = 5241880

# sql_read_query = "SELECT * FROM "+ rearranged_table_name # +" ORDER BY bm25_score"
sql_read_query = "SELECT * FROM "+ table_name +" ORDER BY question , bm25_score"
sql_drop_query = "DROP TABLE " + table_name


con = None
try:
    skip_count = 0
    file_read = 0
    counter = 0
    con = lite.connect(db_path)
    print("SQLite connection established")

    for chunk in pd.read_sql_query(sql=sql_read_query, con=con,chunksize=chunk_size):
        file_read += chunk.shape[0]
        chunk.to_sql(name=rearranged_table_name, con=con, if_exists='append', index=False)
        time_taken = time.clock() - curr_time
        print(file_read," Done\t Time taken : ",(time_taken), "\t ETA : ", (((total_rows-file_read)/chunk_size)*time_taken))
        curr_time = time.clock()

    cursor = con.cursor()
    cursor.execute(sql_drop_query)
    con.commit()

except lite.Error as e:
    print("SQLite Error")
    print(e)
    sys.exit(1)

finally:
    if con:
        con.close()

print("--- %s seconds ---" % (time.clock() - start_time))

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


SQLite connection established


D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


100000  Done	 Time taken :  99.6622572769993 	 ETA :  5124.513674474571
200000  Done	 Time taken :  2.4835912769995048 	 ETA :  125.21969187678262
300000  Done	 Time taken :  2.7161373169983563 	 ETA :  134.22824684127835
400000  Done	 Time taken :  2.803009622002719 	 ETA :  135.71836228582524
500000  Done	 Time taken :  2.9225120909977704 	 ETA :  138.58201634060507
600000  Done	 Time taken :  2.6887253210006747 	 ETA :  124.80740293046611
700000  Done	 Time taken :  2.87968913000077 	 ETA :  130.79202465767898
800000  Done	 Time taken :  2.453117478002241 	 ETA :  108.96453463188594
900000  Done	 Time taken :  3.2924745490017813 	 ETA :  142.95529394819854
1000000  Done	 Time taken :  2.350600484998722 	 ETA :  99.70965185306379
1100000  Done	 Time taken :  2.8552509609980916 	 ETA :  118.26106850338775
1200000  Done	 Time taken :  2.667146195002715 	 ETA :  107.80284862657574
1300000  Done	 Time taken :  2.8539588790008565 	 ETA :  112.49963425955896
1400000  Done	 Time taken :  2.

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [25]:
tart_time = time.clock()
curr_time = time.clock()

chunk_size = 100000 #because 10 answers for a question
bm25_header_names = ["q_id","question","answer","label","a_id","bm25_score"]
trimmed_header_names = ["question","answer","label"]
db_path  = "data.db"
rearranged_table_name = "dataset_rearranged"
selected_table_name = "dataset_selected2"
tsv_op = "../res/dataset.tsv"

total_rows = 5241880

# sql_read_query = "SELECT * FROM "+ rearranged_table_name # +" ORDER BY bm25_score"
sql_read_query = "SELECT * FROM "+ rearranged_table_name# +" ORDER BY question , bm25_score"
sql_drop_query = "DROP TABLE " + rearranged_table_name


con = None
try:
    skip_count = 0
    file_read = 0
    counter = 0
    
    tmp = 0
    con = lite.connect(db_path)
    print("SQLite connection established")

    for chunk in pd.read_sql_query(sql=sql_read_query, con=con,chunksize=chunk_size):
        file_read += chunk.shape[0]
        #if file_read<5200000:
        #  continue
        datalist = []

        for chunklet in np.array_split(chunk, (chunk.shape[0]/10)):
            chunklet = chunklet.reset_index()
            true_index = chunklet.index[chunklet['label'] == 1].tolist()
            for true_id in true_index:
                unicodedata.normalize('NFD', chunklet.iloc[true_id]['question']).encode('ascii', 'ignore')
                unicodedata.normalize('NFD', chunklet.iloc[true_id]['answer']).encode('ascii', 'ignore')
                datalist.append(chunklet.iloc[true_id])
                datalist.append(chunklet.iloc[true_id])
                chunklet = chunklet.drop([true_id])
            chunklet = chunklet.reset_index()
            unicodedata.normalize('NFD', chunklet.iloc[0]['question']).encode('ascii', 'ignore')
            unicodedata.normalize('NFD', chunklet.iloc[0]['answer']).encode('ascii', 'ignore')
            
            unicodedata.normalize('NFD', chunklet.iloc[1]['question']).encode('ascii', 'ignore')
            unicodedata.normalize('NFD', chunklet.iloc[1]['answer']).encode('ascii', 'ignore')
            datalist.append(chunklet.iloc[0])
            datalist.append(chunklet.iloc[-1])
            counter+=10

        selected_chunk = pd.DataFrame(datalist, columns=bm25_header_names)
        del selected_chunk['q_id']
        del selected_chunk['a_id']
        del selected_chunk['bm25_score']
        selected_chunk.to_sql(name=selected_table_name, con=con, if_exists='append', index=False)
        #selected_chunk.to_sql(name=selected_table_name, con=con, if_exists='append', index=False)
        tmp += 1
        #unicodedata.normalize('NFD', selected_chunk).encode('ascii', 'ignore')
#         with open(tsv_op, 'a') as f:
#             print(selected_chunk)
#             selected_chunk.to_csv(f, sep='\t', encoding='utf-8', index=False)

        time_taken = time.clock() - curr_time
        print(file_read," Done\t Time taken : ",(time_taken), "\t ETA : ", (((total_rows-file_read)/chunk_size)*time_taken))
        curr_time = time.clock()
        #if tmp == 2 :
            #break

    # cursor = con.cursor()
    # cursor.execute(sql_drop_query)
    # con.commit()

except lite.Error as e:
    print("SQLite Error")
    print(e)
    sys.exit(1)

finally:
    if con:
        con.close()

print("--- %s seconds ---" % (time.clock() - start_time))

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


SQLite connection established


D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:67: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:69: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


100000  Done	 Time taken :  82.21351294099804 	 ETA :  4227.32017921059
200000  Done	 Time taken :  77.51059171100133 	 ETA :  3907.9910213586336
300000  Done	 Time taken :  81.1687594089999 	 ETA :  4011.262687481484
400000  Done	 Time taken :  77.95635957600098 	 ETA :  3774.553383038476
500000  Done	 Time taken :  75.09470748600143 	 ETA :  3560.900915337204
600000  Done	 Time taken :  68.37652858900037 	 ETA :  3173.9564052670903
700000  Done	 Time taken :  62.35673051199774 	 ETA :  2832.167871778323
800000  Done	 Time taken :  66.49087103799684 	 ETA :  2953.444702462574
900000  Done	 Time taken :  70.5139714379984 	 ETA :  3061.632023072165
1000000  Done	 Time taken :  77.65865742000096 	 ETA :  3294.1870573675365
1100000  Done	 Time taken :  88.07011111099928 	 ETA :  3647.7583180842566
1200000  Done	 Time taken :  72.7315218520016 	 ETA :  2939.720835431682
1300000  Done	 Time taken :  69.14634696199937 	 ETA :  2725.6660216256605
1400000  Done	 Time taken :  70.22421924800074

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:86: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [3]:
# import unicodedata
start_time = time.clock()
curr_time = time.clock()

chunk_size = 100000 #because 10 answers for a question
trimmed_header_names = ["question","answer","label"]
db_path  = "data.db"
rearranged_table_name = "dataset_rearranged"
selected_table_name = "dataset_selected2"
tsv_op = "../res/dataset.tsv"

total_rows = 5241880

# sql_read_query = "SELECT * FROM "+ rearranged_table_name # +" ORDER BY bm25_score"
sql_read_query = "SELECT * FROM "+ selected_table_name# +" ORDER BY question , bm25_score"
sql_drop_query = "DROP TABLE " + rearranged_table_name
encoding = "utf-8"



con = None
try:
    skip_count = 0
    file_read = 0
    counter = 0
    con = lite.connect(db_path)
    print("SQLite connection established")

    for chunk in pd.read_sql_query(sql=sql_read_query, con=con,chunksize=chunk_size):
        file_read += chunk.shape[0]
        datalist = []
        #print(chunk['label'])
        for chunklet in chunk.iterrows():
            chunklet = chunklet[1]
            
            #chunklet['question'] = unicodedata.normalize('NFD', chunklet['question']).decode(encoding).encode('ascii', 'ignore')
            #chunklet['answer'] = unicodedata.normalize('NFD', chunklet['answer']).decode(encoding).encode('ascii', 'ignore')
            chunklet['question'] = unicodedata.normalize('NFD', chunklet['question']).encode('ascii', 'ignore').decode(encoding)
            chunklet['answer'] = unicodedata.normalize('NFD', chunklet['answer']).encode('ascii', 'ignore').decode(encoding)
            #print(chunklet)
            #exit(0)
            datalist.append(chunklet)
            
        selected_chunk = pd.DataFrame(datalist, columns=trimmed_header_names)
        with open(tsv_op, 'a') as f:
            selected_chunk.to_csv(f, sep='\t', encoding='utf-8', index=False)

        time_taken = time.clock() - curr_time
        print(file_read," Done\t Time taken : ",(time_taken), "\t ETA : ", (((total_rows-file_read)/chunk_size)*time_taken))
        curr_time = time.clock()

    # cursor = con.cursor()
    # cursor.execute(sql_drop_query)
    # con.commit()

except lite.Error as e:
    print("SQLite Error")
    print(e)
    sys.exit(1)

finally:
    if con:
        con.close()

print("--- %s seconds ---" % (time.clock() - start_time))

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


SQLite connection established


D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


100000  Done	 Time taken :  19.181239968 	 ETA :  986.2763416665983
200000  Done	 Time taken :  19.50340294700004 	 ETA :  983.3381725042055
300000  Done	 Time taken :  18.637950505999925 	 ETA :  921.065148465909
400000  Done	 Time taken :  18.874330501999907 	 ETA :  913.872433710233
500000  Done	 Time taken :  19.54075362900005 	 ETA :  926.5990881828276
600000  Done	 Time taken :  19.675239264000083 	 ETA :  913.3009963477671
700000  Done	 Time taken :  19.042413286000055 	 ETA :  864.8835605541793
800000  Done	 Time taken :  18.633305128000075 	 ETA :  827.6690538196096
900000  Done	 Time taken :  19.03818664500011 	 ETA :  826.6152183019307
1000000  Done	 Time taken :  19.025916397999936 	 ETA :  807.0565425034796
1100000  Done	 Time taken :  18.870643802000018 	 ETA :  781.5994215062783
1200000  Done	 Time taken :  18.939904221999996 	 ETA :  765.5282007681734
1300000  Done	 Time taken :  18.917421168000033 	 ETA :  745.7020415371596
1400000  Done	 Time taken :  18.7940167769999

D:\WindowsSoftware\Anaconda\envs\TMINEFM\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
